In [ ]:
# Notebook: 2_bronze_to_silver
bronze_prefix = "dbfs:/mnt/datalake/bronze"
silver_prefix = "dbfs:/mnt/datalake/silver"

movies = spark.read.format("delta").load(f"{bronze_prefix}/movies_top100")
ratings = spark.read.format("delta").load(f"{bronze_prefix}/movies_ratings")
revenue = spark.read.format("delta").load(f"{bronze_prefix}/movies_revenue")
casts = spark.read.format("delta").load(f"{bronze_prefix}/movies_casts")
meta = spark.read.format("delta").load(f"{bronze_prefix}/movies_meta")

from pyspark.sql.functions import col, trim
movies = movies.withColumn("Year", trim(col("Year")).cast("int"))
ratings = ratings.withColumn("IMDb_Rating", col("IMDb_Rating").cast("double"))

silver = (movies.join(ratings, ["Title"], "left")
    .join(revenue, ["Title"], "left")
    .join(casts, ["Title"], "left")
    .join(meta, ["Title"], "left")
    .dropDuplicates(["Title"]))
silver.write.format("delta").mode("overwrite").option("overwriteSchema","true").partitionBy("Year").save(f"{silver_prefix}/top100_combined")
print("silver written")
